# Transaction Volumes & Velocity Algorithm for Calculations

This iPython script contains the algorithm used to parse BitShares's blockchain for the trading history for the top 250 BTS token holders in order to estimate the velocity of money for BTS tokens in the past year. A list of top BTS holders is available at http://cryptofresh.com/a/BTS. The file ("BTS_top_holders.csv") shown in script below contains the list of top BTS holders as of 17 February 2018.


In [1]:
import pandas as pd
from datetime import datetime, timedelta, date
from bts_blockchain_parser import History
from bitshares.account import Account

history = History()

def transactions(account_name):
    # This function parses the BitShares blockchain for the BTS transfers and filled orders that 
    # took place in the past year and aggregates the transaction volume.
        
    account = Account(account_name)
    volume = 0
    
    # The block number that represents 1 year ago (so we'll know how far to go back).
    block_num_threshold = history.block_search(datetime.now() - timedelta(days = 365))
    for trade in account.history(only_ops = ["transfer", "fill_order"]):
        if trade.get("block_num") > block_num_threshold:
            if "amount" in trade["op"][1]:
                if trade["op"][1]["amount"].get("asset_id") == "1.3.0":
                    volume = volume + int(trade["op"][1]["amount"].get("amount"))
            else:
                pass
    return (volume/100000)


top_accounts = pd.read_csv("BTS_top_holders.csv", index_col = ["Rank"])
top_accounts["Transaction Volume"] = top_accounts["Account"].apply(transactions)
top_accounts["Transaction Volume"].astype(int)
top_accounts["Velocity"] = top_accounts["Transaction Volume"]/top_accounts["Holdings"]

In [2]:
top_accounts.head()

,Account,Balance,Collateral,Holdings,Transaction Volume,Velocity
Rank,,,,,,
1,poloniexcoldstorage,230001204,-,230001204,6.000000e+07,0.260868
2,binance-cold-1,114787664,-,114787664,1.147877e+08,1.000000
3,binance-bitshares-gvbdb84k6h,82097772,-,82097772,1.153550e+06,0.014051
4,zbbts,77720941,-,77720941,2.332845e+04,0.000300
5,delta-lion,916,72832248,72833164,0.000000e+00,0.000000


In [3]:
top_accounts.to_csv("top_holders_list.csv")